In [1]:
import game
import time
import copy
import pygame
import random
import gamestate
import constants
import components
import numpy as np

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
g = game.Game()
g.update_ui()

In [3]:
%%time
while not g.is_done():
    g.game_state.random_move()
    g.update_ui()
# time.sleep(5)
# pygame.quit()

CPU times: user 1.36 s, sys: 156 ms, total: 1.52 s
Wall time: 4.29 s


In [66]:
g.game_state.restart()
g.update_ui()

restart


In [4]:
def random_ai_move(game_: game.Game, searches_per_move=10, search_length=10, update_ui=False):
    scores = np.zeros(4)
    game_state_init_copy = game_.game_state.copy()
    for first_move_i in range(4):
        game_.set_game_state(game_state_init_copy)
        first_move = constants.ARROW_KEYS[first_move_i]
        score, changed = game_.game_state.move(first_move)
        game_state_tmp1_copy = game_.game_state.copy()
        if update_ui:
            game_.update_ui()
        # time.sleep(1)
        if changed:
            scores[first_move_i] += score
        else:
            continue
        max_cumulative_score = 0
        for later_moves in range(searches_per_move):
            move_number = 1
            game_.set_game_state(game_state_tmp1_copy)
            changed = True
            score_cumulative = 0
            while changed and move_number < search_length:
                score, changed = game_.game_state.random_move()
                if update_ui:
                    game_.update_ui()
                if changed:
                    score_cumulative += score
                    scores[first_move_i] += score
                    move_number += 1
            max_cumulative_score = max(max_cumulative_score, score_cumulative)
        # scores[first_move_i] += max_cumulative_score
    # restore original game state
    game_.set_game_state(game_state_init_copy)
        
    best_move_index = np.argmax(scores)
    best_move = constants.ARROW_KEYS[best_move_index]
    return best_move
    

In [8]:
%%time
seed_performance_dict = {}
for seed in range(100):
    t_0 = time.time()
    print(f"seed {seed}")
    constants.RANDOM_SEED = seed
    g.game_state.restart()
    g.update_ui()
    for i in range(1500):
        move = random_ai_move(g, searches_per_move=20, search_length=10)
        g.game_state.move(move)
        g.update_ui()
        if g.game_state.is_done:
            max_val = np.max(g.game_state.get_board().get_value_board())
            print(f"done, {i} iters, time: {round(time.time() - t_0, 2)}s, max_val: {max_val}")
            g.update_ui()
            seed_performance_dict[seed] = max_val
            break
        if 2048 in g.game_state.get_board().get_value_board():
            print("success")
            break


ime: 28.2s, max_val: 512
done, 630 iters, time: 28.27s, max_val: 512
done, 631 iters, time: 28.34s, max_val: 512
done, 632 iters, time: 28.4s, max_val: 512
done, 633 iters, time: 28.47s, max_val: 512
done, 634 iters, time: 28.53s, max_val: 512
done, 635 iters, time: 28.61s, max_val: 512
done, 636 iters, time: 28.67s, max_val: 512
done, 637 iters, time: 28.74s, max_val: 512
done, 638 iters, time: 28.8s, max_val: 512
done, 639 iters, time: 28.87s, max_val: 512
done, 640 iters, time: 28.93s, max_val: 512
done, 641 iters, time: 29.0s, max_val: 512
done, 642 iters, time: 29.07s, max_val: 512
done, 643 iters, time: 29.13s, max_val: 512
done, 644 iters, time: 29.2s, max_val: 512
done, 645 iters, time: 29.27s, max_val: 512
done, 646 iters, time: 29.33s, max_val: 512
done, 647 iters, time: 29.4s, max_val: 512
done, 648 iters, time: 29.46s, max_val: 512
done, 649 iters, time: 29.53s, max_val: 512
done, 650 iters, time: 29.59s, max_val: 512
done, 651 iters, time: 29.66s, max_val: 512
done, 652 it

KeyboardInterrupt: 

In [67]:
%%time
constants.RANDOM_SEED = 2048
g.game_state.restart()
g.update_ui()
for i in range(50):
    move = random_ai_move(g, searches_per_move=20, search_length=10)
    g.game_state.move(move)
    g.update_ui()
    if g.game_state.is_done:
        max_val = np.max(g.game_state.get_board().get_value_board())
        print(f"done, {i} iters, max_val: {max_val}")
        g.update_ui()
        seed_performance_dict[seed] = max_val
        break
    if 2048 in g.game_state.get_board().get_value_board():
        print(f"success in {i} moves")
        break

restart
CPU times: user 1.99 s, sys: 0 ns, total: 1.99 s
Wall time: 2 s


In [72]:
matrix = [
    [None, None, None, 2],
    [2, None, None, 2],
    [None, None, None, None],
    [None, None, None, None]
]

In [73]:
%%time
game_ = gamestate.GameState(matrix=matrix)

CPU times: user 104 µs, sys: 39 µs, total: 143 µs
Wall time: 149 µs


In [74]:
%%time
game_.move(constants.RIGHT)

CPU times: user 75 µs, sys: 28 µs, total: 103 µs
Wall time: 107 µs


(4, True)

In [78]:
%%time
game_.compute_is_done()

CPU times: user 100 µs, sys: 38 µs, total: 138 µs
Wall time: 140 µs


False

In [41]:
print(game_._board)

      0     1     2     3
0  None  None  None     2
1  None  None  None     4
2  None  None     2     2
3  None  None  None  None


In [17]:
2048 in g.game_state.get_board().get_value_board()

True

In [7]:
np.max(g.game_state.get_board().get_value_board())

1024

In [10]:
max([seed_performance_dict[key] for key in seed_performance_dict.keys()])

2048

In [11]:
seed_performance_dict

{0: 512,
 1: 1024,
 2: 1024,
 3: 512,
 4: 1024,
 5: 1024,
 6: 512,
 7: 1024,
 8: 1024,
 9: 256,
 10: 1024,
 11: 1024,
 12: 512,
 13: 256,
 14: 512,
 15: 512,
 16: 1024,
 17: 512,
 18: 512,
 19: 1024,
 20: 512,
 21: 1024,
 22: 512,
 23: 1024,
 24: 512,
 25: 512,
 26: 512,
 27: 1024,
 28: 512,
 29: 512,
 30: 512,
 31: 512,
 32: 512,
 33: 512,
 34: 512,
 35: 1024,
 36: 1024,
 37: 1024,
 38: 512,
 39: 512,
 40: 1024,
 41: 1024,
 42: 512,
 43: 512,
 44: 512,
 45: 512,
 46: 512,
 47: 256,
 48: 512,
 49: 1024,
 50: 1024,
 51: 512,
 52: 1024,
 53: 1024,
 54: 512,
 55: 256,
 56: 1024,
 57: 512,
 58: 1024,
 59: 512,
 60: 512,
 61: 1024,
 62: 1024,
 63: 1024,
 64: 512,
 65: 512,
 66: 1024,
 67: 256,
 68: 1024,
 69: 1024,
 70: 1024,
 71: 1024,
 72: 512,
 73: 512,
 74: 1024,
 75: 512,
 76: 512,
 77: 512,
 78: 512,
 79: 1024,
 80: 512,
 81: 512,
 82: 1024,
 83: 512,
 84: 1024,
 85: 1024,
 86: 2048,
 87: 1024,
 88: 512,
 89: 512,
 90: 1024,
 91: 1024,
 92: 1024,
 93: 512,
 94: 512,
 95: 1024,
 96: 25